In [1]:
# Notes: 'No. of Rows' pertains to the no. of rows in the initial Outlook email

import os
import re
import pandas as pd
import win32com.client
import openpyxl


class MiscFunctions:

    def closeExcel(self):
        excel = win32com.client.Dispatch('Excel.Application')
        wb = excel.Workbooks.Add()
        excel.DisplayAlerts = False
        wb.Close(True)
        excel.Application.Quit()
        excel.DisplayAlerts = True
    
    def emptyOutputsFolder(self):
        list_outputs_folder = os.listdir(self.folder_outputs_url)
        if len(list_outputs_folder) >= 1:
            for i in list_outputs_folder:
                os.remove("{0}\{1}".format(self.folder_outputs_url, i))

class UserInputs(MiscFunctions):

    def __init__(self, inputs_folder_name=None, outputs_folder_name=None):
        self.folder_inputs_url = os.path.join('..', inputs_folder_name)
        self.folder_outputs_url = os.path.join('..', outputs_folder_name)
        
        self.length_folder_inputs = len(os.listdir(self.folder_inputs_url))
        self.length_folder_outputs = len(os.listdir(self.folder_outputs_url))

class OrderForm(UserInputs):
    
    def getEmails(self) -> list:
        list_emails = os.listdir(self.folder_inputs_url)
        return list_emails
    
    def standardizeDates(self):
        list_dates = []
        
        for i in range(self.length_folder_inputs):
            # extract: date pattern from the list of emails in '(1) Inputs' folder. 
            # extract: capture the 1st occurance; hence [0] applied
            date = re.findall(r'\d{1,2}[_ ]\d{1,2}[_ ]\d{4}', self.getEmails()[i])[0]
            # clean: substitute underscores or blanks with '/'
            # clean: returns -> <m>m/<d>d/yyyy
            date = re.sub(r'_| ', "/", date)
            # breakout: creates a tuple and delineates by "/"
            # breakout: returns -> [('<m>m', '<d>d', '<yyyy>')]
            # notes: for some reason findall will return a list; hence [0] applied
            date = re.findall(r'(\d{1,2})[\/](\d{1,2})[\/](\d{4})', date)[0]
            # Create: obj
            mm = date[0]
            dd = date[1]
            yyyy = date[2]
        
            # Additional mm/dd standardization
            if len(mm) == 1:
                mm = "0" + mm

            if len(dd) == 1:
                dd = "0" + dd

            # Final: Date standardization
            date = "{}-{}-{}".format(yyyy,mm,dd)

            # Append: Final to date_list
            list_dates.append(
                date)

        return list_dates

    def standardizedEnvironmentNames(self) -> list:
        
        list_environment_names = []
        
        for i in range(self.length_folder_inputs):
            target = self.getEmails()
            
            if "Prod" in target[i]:
                list_environment_names.append("Production")
            elif "Dev" in target[i]:
                list_environment_names.append("Development")

        return list_environment_names
    
    def getAbsEmailPaths(self):
        list_abs_email_paths = []

        # Take Aways: "{0}\\{1}".format(self.folder_inputs_url, self.getEmails()
        # Take Aways: self.getEmails()
        # Take Aways: self was needed infront of .getEmails()
        # Take Aways: However, there is no self in .getEmails(<self - there is none>)
        for i in range(self.length_folder_inputs):
            abs_email_path = os.path.abspath("{0}\\{1}".format(self.folder_inputs_url, self.getEmails()[i]))
            list_abs_email_paths.append(abs_email_path)
        
        return list_abs_email_paths

    def orderForm(self):
        dict_order_form = {}
        list_order_form = []
        
        for i in range(self.length_folder_inputs):
            dict_order_form = {
                "file_name": self.getEmails()[i],
                "date": self.standardizeDates()[i],
                "environment": self.standardizedEnvironmentNames()[i],
                "abs_email_path": self.getAbsEmailPaths()[i]
            }
            
            list_order_form.append(dict_order_form)

        df = pd.DataFrame(list_order_form, columns=['file_name', 'date', 'environment', 'abs_email_path'])
        df = df.sort_values(["date", "environment"], ascending=[True,False])
        df = df.reset_index(drop=True)

        return df
    
class ScrapeEmails(OrderForm):
    
    def scrapeOutlookEmail(self, url_email):

        outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
        msg = outlook.OpenSharedItem(url_email)
        
        # counter - Rows(\r) from 'Body'
        self.num_of_rows = re.findall("\r", msg.Body)

        # list - Cells(\t) from 'Body'
        self.list_outlook_cells = re.split(r'\t', msg.Body)
        
        # scrape - Subject Line
        subject = msg.Subject      

        ### START Scrape - Date
        date = re.findall(r'(\d+/\d+/\d+)', subject)[0]
        # Used to breakout via tuple[(MM, DD, YYYY)]
        date = re.findall(r'(\d{1,2})[\/](\d{1,2})[\/](\d{4})', date)[0]

        # Additional 'mm/dd/yy standardization
        mm = date[0]
        dd = date[1]
        yy = date[2][2:4]

        if len(mm) == 1:
            mm = "0" + mm

        if len(dd) == 1:
            dd = "0" + dd

        self.date = "'{}/{}/{}".format(mm,dd,yy)
        ### END Scrape - Date

        ### START: Scrape - Environment
        environment = None

        if "Pro" in subject:
            self.environment = "Production"
        elif "Dev" in subject:
            self.environment = "Development"
        ### END: Scrape - Environment
        
        print("Processing      : " + "{0} of {1}".format(i+1, self.length_folder_inputs))
        print("Name of File    : " + self.orderForm()['file_name'][i])
        print("Date            : " + self.orderForm()['date'][i])
        print("Environment     : " + self.orderForm()['environment'][i])
        print("No. of Rows     : " + str(len(self.num_of_rows)))
        print("Scrape Status   : " + "Completed")
        
    def exportToOpenpyxl(self):

        wb = openpyxl.Workbook()
        sheet = wb['Sheet']

        # Counters   
        rowNumber = 1
        columnNumber = 0

        # Copy and Paste: Outlook object to OpenPy
        # For number of cells (\t) in the outlook object
        for i in range(len(self.list_outlook_cells)):
            # If the cells obj at the time of index contains '\r' ('\r' indicates a new row)...
            # If \r = TRUE
            if len(re.findall('\r', self.list_outlook_cells[i])) > 0:
                # Add to the row counter to designate the current row...
                rowNumber = rowNumber + len(re.findall('\r', self.list_outlook_cells[i]))
                # Then reset the column counter back to 1 ...
                columnNumber = 1
                sheet.cell(row=rowNumber, column=columnNumber).value = self.list_outlook_cells[i].strip()

            else:
                # Otherwise keep copying & pasting right
                columnNumber = columnNumber + 1
                sheet.cell(row=rowNumber, column=columnNumber).value = self.list_outlook_cells[i].strip()

        # Step 2: Adding 4 columns
        sheet.insert_cols(1,amount=4)

        # Step 3: Adding the A, B, C & D formulas. Then applying an array function
        for i in range(len(self.num_of_rows)-1):
            sheet['A{0}'.format(i+2)] = '=NOT(OR("State: "=LEFT(E{0},7),E{0}="AVM",E{0}=""))'.format(i+2)
            sheet['B{0}'.format(i+2)] = self.environment
            sheet['C{0}'.format(i+2)] = self.date
            sheet['D{0}'.format(i+2)] = '=TRIM(MID(INDEX(E:E,MAX(IF("State: "=LEFT($E$2:E{0},7),ROW($E$2:E{0}),))),7,10))'.format(i+2)

            # Shift + Control + Enter
            wb['Sheet'].formula_attributes['D' + str(i+2)] = {'t': 'array', 'ref': 'D' + str(i+2) + ':D' + str(i+2)}

        # Step 4: Saving the file
        wb.save(self.folder_outputs_url + "\Temp.xlsx")
        
        print("OpenPyXl Status : " + "Completed")

    def executeCopyAndPasteAsValuesToWin32(self):
        """
        Win32 is needed to "paste as values"
        """

        excel = win32com.client.Dispatch('Excel.Application')
        excel.Visible = False

        w32wb1 = excel.Workbooks.Open(os.path.abspath(self.folder_outputs_url + "\Temp.xlsx"))
        w32ws1 = excel.ActiveSheet

        w32wb2 = excel.Workbooks.Add()
        w32ws2 = excel.ActiveSheet
        excel.Visible = False

        currentCell = 1

        # For every true value
        for i in range(1, len(self.num_of_rows)+1):
        # Copy and paste content to a new sheet.
        # This method just avoids a double for loop
            if w32ws1.Cells(i, 1).Value == True:
                w32ws2.Cells(currentCell, 1).Value = w32ws1.Cells(i, 2).Value
                w32ws2.Cells(currentCell, 2).Value = w32ws1.Cells(i, 3).Value
                w32ws2.Cells(currentCell, 3).Value = w32ws1.Cells(i, 4).Value
                w32ws2.Cells(currentCell, 4).Value = w32ws1.Cells(i, 5).Value
                w32ws2.Cells(currentCell, 5).Value = w32ws1.Cells(i, 6).Value
                w32ws2.Cells(currentCell, 6).Value = w32ws1.Cells(i, 7).Value
                w32ws2.Cells(currentCell, 7).Value = w32ws1.Cells(i, 8).Value
                w32ws2.Cells(currentCell, 8).Value = w32ws1.Cells(i, 9).Value
                w32ws2.Cells(currentCell, 9).Value = w32ws1.Cells(i, 10).Value
                w32ws2.Cells(currentCell, 10).Value = w32ws1.Cells(i, 11).Value
                w32ws2.Cells(currentCell, 11).Value = w32ws1.Cells(i, 12).Value
                w32ws2.Cells(currentCell, 12).Value = w32ws1.Cells(i, 13).Value
                w32ws2.Cells(currentCell, 13).Value = w32ws1.Cells(i, 14).Value
                w32ws2.Cells(currentCell, 14).Value = w32ws1.Cells(i, 15).Value
                w32ws2.Cells(currentCell, 15).Value = w32ws1.Cells(i, 16).Value
                w32ws2.Cells(currentCell, 16).Value = w32ws1.Cells(i, 17).Value

                currentCell += 1

        # This needs to be here because you must close the file first before you overwrite it
        w32wb1.Close(True)

        excel.DisplayAlerts = False
        w32ws2.SaveAs(os.path.abspath(self.folder_outputs_url + "\Temp.xlsx"))
        excel.DisplayAlerts = True

        w32wb2.Close(True)
        excel.Application.Quit()

        print("Win32 Status    : " + "Completed")

    def appendDataUsingPandas(self):

        def append(self):
            # Append: data to the file
            read_in = pd.read_excel(self.folder_outputs_url + "\Temp.xlsx",
                                    header=None)
            # Notes: Do not do this
            # read_in[1] = read_in[1].map(lambda x: "'" + str(x) if x else x)
            append_to = pd.read_excel(self.folder_outputs_url + "\Preprocessing.xlsx", header=None)
            df_append = append_to.append(read_in, ignore_index=True)
            df_append.to_excel(self.folder_outputs_url + "\Preprocessing.xlsx",
                               sheet_name = "Data Warehouse - VTS Monitoring",
                               index=False,
                               header=False)

        # If: Preprocessing.xlsx exists
        if "Preprocessing.xlsx" in os.listdir(self.folder_outputs_url):
            append(self)

        else:
            # Create: Preprocessing.xlsx
            df_preprocessing = pd.DataFrame()
            df_preprocessing.to_excel(self.folder_outputs_url + "\Preprocessing.xlsx", index=False, header=False)
            append(self)

        # Delete: The temp file
        os.remove(self.folder_outputs_url + "\Temp.xlsx")

        print("Pandas Status   : " + "Completed")
        print("-" * 80)

    def appendToDailyDbOne(self):
        file_DbOne = r"C:\Users\Reuben\DbOne.csv"
        file_preprocessing = self.folder_outputs_url + "\Preprocessing.xlsx"

        # Create: DF object 1
        df_DbOne = pd.read_csv(file_DbOne)
        headers_df_DbOne = list(df_DbOne.columns)

        # Create: DF object 2
        df_preprocessing = pd.read_excel(file_preprocessing, header=None, names=headers_df_DbOne)

        # Append: DF's
        df_append = df_DbOne.append(df_preprocessing, ignore_index=True)

        # Save: file
        df_append.to_csv(file_DbOne, index=False, header=True)
    






##### Execute ####################################################################################################




a = ScrapeEmails(inputs_folder_name="(1) Inputs", outputs_folder_name="(3) Outputs")

a.closeExcel()
a.emptyOutputsFolder()

for i in range(a.length_folder_inputs):
    a.scrapeOutlookEmail(a.orderForm()['abs_email_path'][i])
    a.exportToOpenpyxl()
    a.executeCopyAndPasteAsValuesToWin32()
    a.appendDataUsingPandas()

print("Preprocessing.xlsx file created. Exporting to VTS DB, please wait...")
print("=" * 80)

# To go straight to 'Preprocessing.xlsx' comment out function below
a.appendToDailyDbOne()

print("PROGRAM COMPLETED")

Processing      : 1 of 2
Name of File    : Production VTS Results for 4_16_2019 through 4_16_2019.msg
Date            : 2019-04-16
Environment     : Production
No. of Rows     : 839
Scrape Status   : Completed
OpenPyXl Status : Completed
Win32 Status    : Completed
Pandas Status   : Completed
--------------------------------------------------------------------------------
Processing      : 2 of 2
Name of File    : Dev VTS Results for 4_16_2019 through 4_16_2019.msg
Date            : 2019-04-16
Environment     : Development
No. of Rows     : 2554
Scrape Status   : Completed
OpenPyXl Status : Completed
Win32 Status    : Completed
Pandas Status   : Completed
--------------------------------------------------------------------------------
Preprocessing.xlsx file created. Exporting to VTS DB, please wait...
PROGRAM COMPLETED
